In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-04-15 11:41:15,512 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.2.0.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/working


In [6]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()}

graphnet: INFO     2023-04-15 11:41:22 - get_logger - Writing log to logs/graphnet_20230415-114122.log
graphnet: WARNING  2023-04-15 11:41:22 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-04-15 11:41:25.362475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 11:41:26.125610: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different nume

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Predicting DataLoader 0: 100%|██████████████| 6250/6250 [03:43<00:00, 27.91it/s]
2023-04-15 12:10:41,674 [INFO][11-inference-parquet] Num of ensemble: 8.
2023-04-15 12:10:44,632 [INFO][11-inference-parquet] Make submission.
2023-04-15 12:10:47,275 [INFO][11-inference-parquet] results columns: Index(['event_id', 'direction_x', 'direction_y', 'direction_z',
       'direction_kappa', 'zenith_x', 'azimuth_x', 'sigma', 'azimuth_y',
       'zenith_y'],
      dtype='object')
2023-04-15 12:10:47,275 [INFO][11-inference-parquet] submission columns: Index(['azimuth', 'zenith'], dtype='object')
2023-04-15 12:10:47,275 [INFO][11-inference-parquet] Done.


In [7]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-04-15 12:10:53,575 [INFO][utils] Output directory: output/2023-04-15_12-10-53
2023-04-15 12:10:53,575 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-04-15 12:10:53,575 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-04-15 12:10:53,576 [INFO][utils] Fix seed: 440
2023-04-15 12:10:53,577 [INFO][15-validation] Validation stage 1.
2023-04-15 12:10:53,945 [INFO][15-validation] Base score: 1.0138796748577477
2023-04-15 12:10:53,956 [INFO][15-validation] Num of low sigma events: 52777, Num of high sigma events: 147223
2023-04-15 12:10:54,004 [INFO][15-validation] Low sigma score: 0.2117962987833093, High sigma score: 1.3014132418892617
2023-04-15 12:10:54,004 [INFO][15-validation] Done.


In [8]:
# !python 11-inference-parquet.py settings.in_kaggle={in_kaggle()} training_params.stage2=True global_params.seed=2440

In [9]:
# !python 15-validation.py settings.in_kaggle={in_kaggle()} training_params.stage2=True

In [10]:
# !mv results.csv results_stage2.csv

In [11]:
# !python 11-inference-parquet.py settings.in_kaggle={in_kaggle()} inference_params.model_path=icecube/model_1_100.pth global_params.seed=1440

In [12]:
# !python 15-validation.py settings.in_kaggle={in_kaggle()}

In [13]:
# !python 20-ensemble.py settings.in_kaggle={in_kaggle()}